## QuickStart from pytorch_tutorials

In [2]:
"""导包"""
import torch
from torch import nn
# torch.utils.data.Dataset和torch.utils.data.DataLoader是pytorch的两大关键原语
# Dataset存储样本和对应标签，DataLoader将Dataset包装成一个可迭代的对象
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
"""下载数据集"""
# torchvision.datasets包含很多真实世界的数据集如CIFAR、COCO
# 本例中使用FashionMNIST数据集
# 每个torchvision的数据集都包含两个参数，transform和target_transform，分别修改样本和标签。

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

26422272it [00:03, 7852378.84it/s]                               


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



29696it [00:00, 136812.60it/s]                           


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



4422656it [00:02, 1905888.36it/s]                             


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



6144it [00:00, 14120440.43it/s]         

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw




/Users/zhongshannan/opt/anaconda3/envs/Py38/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
"""创建dataloader"""
# 将Dataset作为参数传递给DataLoader
# 对数据集包装成迭代，以支持自动批处理、采样、重组和多进程数据加载
# 设置batch size为64，即每次DataLoader迭代会返回一组64个的特征和标签

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [7]:
for X, y in test_dataloader:
    print(f"X.shape:{X.shape}")
    print(f"y.shape:{y.shape}")
    break ## 印证batch_size=64

X.shape:torch.Size([64, 1, 28, 28])
y.shape:torch.Size([64])


## 创建模型

In [9]:
# 选择gpu/cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用:{device} device")

使用:cpu device


In [11]:
# 定义模型
class Net(nn.Module):
    def __init__(self):  # 定义__init__方法
        super(Net, self).__init__() # 继承Net的父类nn.module中的方法
        self.flatten = nn.Flatten() # 创建flatten
        self.linear_relu_stack = nn.Sequential(  # 创建linear_relu_stack
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):  # 向前传播
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = Net().to(device) # 将模型copy到device上一份
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 优化模型参数

In [12]:
# 训练模型需要一个损失函数和一个优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## 训练，反向传播

In [13]:
# 在单次训练循环中，模型对训练数据集进行预测
# 按照batch_size输入至模型中，并反向传播预测误差以调整模型的参数
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) 
    model.train() # 训练模式
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) # 将数据copy到device上一份

        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)
    
        #反向传播
        optimizer.zero_grad() # 梯度清零
        loss.backward() # loss自动取得平均
        optimizer.step()

        if batch % 100 == 0: # 每100个batch输出
            loss, current = loss.item(), batch*len(X) #batch*batch_size
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5f}]")

In [14]:
# 测试模型性能
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batchs = len(dataloader) # 测试batch数量,为了求平均loss
    model.eval() # 评估模式
    test_loss, correct = 0, 0
    with torch.no_grad(): # 测试阶段无梯度参与,模型固定
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()  #loss手动取平均
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 统计预测准确的个数
    test_loss /= num_batchs
    correct /= size
    print(f"Test Erorr: \n Acc:{(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 输出结果

In [15]:
# 训练经过多个迭代过程，合并视为1个epochs
# 分别打印出每个epoch的准确率增加和损失减少
epochs = 5
for t in range(epochs):
    print(f"Epoch{t+1}\n------------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Finish!")

Epoch1
------------------
loss: 2.303030 [    0/60000.000000]
loss: 2.289198 [ 6400/60000.000000]
loss: 2.266241 [12800/60000.000000]
loss: 2.256635 [19200/60000.000000]
loss: 2.242709 [25600/60000.000000]
loss: 2.204225 [32000/60000.000000]
loss: 2.215483 [38400/60000.000000]
loss: 2.178748 [44800/60000.000000]
loss: 2.176834 [51200/60000.000000]
loss: 2.135964 [57600/60000.000000]
Test Erorr: 
 Acc:51.3%, Avg loss: 2.135379 

Epoch2
------------------
loss: 2.145486 [    0/60000.000000]
loss: 2.140833 [ 6400/60000.000000]
loss: 2.076876 [12800/60000.000000]
loss: 2.094315 [19200/60000.000000]
loss: 2.040385 [25600/60000.000000]
loss: 1.969397 [32000/60000.000000]
loss: 2.004328 [38400/60000.000000]
loss: 1.915445 [44800/60000.000000]
loss: 1.929653 [51200/60000.000000]
loss: 1.842666 [57600/60000.000000]
Test Erorr: 
 Acc:55.0%, Avg loss: 1.847014 

Epoch3
------------------
loss: 1.882837 [    0/60000.000000]
loss: 1.858758 [ 6400/60000.000000]
loss: 1.731044 [12800/60000.000000]
lo

## 保存模型

In [16]:
# 序列化模型内部状态字典，包含模型参数
torch.save(model.state_dict(), "model.pth")
print("模型保存至model.pth")

模型保存至model.pth


## 加载模型

In [17]:
# 重新创建模型结构，并将状态字典加载到模型中
model = Net()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [20]:
# 利用加载模型进行一些预测
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # 评估模式
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted = classes[pred[0].argmax(0)] # .argmax(0)取一行中最大的值
    actual = classes[y]
    print(pred)
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([[-1.9601, -2.4649, -0.9004, -1.6704, -1.1302,  2.3281, -1.0269,  2.5501,
          1.7633,  2.9714]])
Predicted: "Ankle boot", Actual: "Ankle boot"
